# BDA project "Pump It Up"

- Анастасия Барковая
- Варвара Михайлова
- Анна Снигирева

# 1. Проблема

В соответствии с [water.org ](https://water.org ), 4 миллиона человек в Танзании не имеют доступа к улучшенному источнику безопасной воды.

Люди, живущие в таких условиях, тратят значительное количество времени на поездки на большие расстояния, чтобы набрать воды в пунктах сбора воды.

Точное предсказание того, функционируют насосы или нет, действительно важно, поскольку люди подвергаются риску ухудшения здоровья или даже смерти от обезвоживания.

![image.png](attachments/image.png)

Используя данные Taarifa и Министерства водного хозяйства Танзании, мы должны понимать, какие насосы исправны, какие нуждаются в некотором ремонте, а какие вообще не работают. Прогноз будет основан на ряде переменных, касающихся того, какой тип насоса работает, когда он был установлен и как им управляют. Четкое понимание того, какие пункты водоснабжения выйдут из строя, может улучшить операции по техническому обслуживанию и обеспечить доступность чистой питьевой воды для населения по всей Танзании.

[Ссылка на конкурс](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table )

# 2. Подгрузка датасета

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
X = pd.read_csv("dataset/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv", index_col=0)
y = pd.read_csv("dataset/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv", index_col=0)
assert (X.index == y.index).all()

In [4]:
pd.options.display.max_columns = 41
X.head(5)

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


Посмотрим, сколько всего количественных и категориальных признаков

In [5]:
cat_features = 0
numeric_features = 0
for column, tp in X.dtypes.items():
    if str(tp) == 'object':
        cat_features += 1
    else:
        numeric_features += 1
print(f'There are {cat_features} categorial features and {numeric_features} numeric features.')

There are 30 categorial features and 9 numeric features.


Посмотрим, сколько N/A признаков

In [6]:
y.isna().sum()

status_group    0
dtype: int64

In [7]:
categorial_nan_count = X.select_dtypes(include=['object']).isna().sum()
numeric_nan_count = X.select_dtypes(exclude=['object']).isna().sum()
#nan_count.combine(X.dtypes)
#types = X.dtypes
print(f'NaN in Categorial:\n{categorial_nan_count}')
print(f'NaN in Numerics:\n{numeric_nan_count}')


NaN in Categorial:
date_recorded                0
funder                    3635
installer                 3655
wpt_name                     0
basin                        0
subvillage                 371
region                       0
lga                          0
ward                         0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity                     0
quantity_group               0
source                       0
source_type                  0
source_class                 0
waterpoint_type              0
waterpoint_type_group        0
dtype: int64
NaN in Numerics:
amount_tsh           

Как видим, NaN встречается только в категориальных признаках, поэтому заменим строкой "None"

In [8]:
X.fillna(value="None", inplace=True)